In [ ]:
# Beräknar antalet ratings per film
rating_per_movie = df_ratings.groupby('movieId').size().reset_index(name='num_ratings')
# Sorterar filmer efter nr ratings
sorted_movies = rating_per_movie.sort_values(by='num_ratings', ascending=False)
# 10 filmer med flest ratings
top_10_movies = sorted_movies.head(10)
print('Top 10 Movies with Most Ratings')
print(top_10_movies)

In [ ]:
def recommender(movie_name, model, n_recommendations):
    idx = process.extractOne(movie_name, df_movies["title"])[2]
    print("Movie Selected: ", df_movies["title"][idx], "Index: ", idx)
    print("Searching for recommendations....")
    distances, indices = model.kneighbors(sparse_matrix[idx].reshape(1, -1), n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i != idx))

recommender("fight club", model_knn,20)

In [ ]:
# Först, fyll NaN-värden med en tom sträng i kolumnen 'tag'
df_tags_filled = df_tags.fillna({'tag': ''})

# Filtrera filmer som tillhör en specifik genre
desired_genre = "Drama"  # Till exempel, vi väljer genren Drama
df_movies_genre = df_tags_filled[df_tags_filled['tag'].str.contains(desired_genre, case=False)]

# Skapa en ny DataFrame för betyg för filmer som tillhör den specifika genren
df_filtered_genre = df_ratings[df_ratings['movieId'].isin(df_movies_genre['movieId'])]

# Träna modellen och göra rekommendationer som vanligt
model_knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=2)
model_knn.fit(df_filtered_genre)  # Träna modellen med data från den specifika genren

def recommender_genre(movie_name, model, n_recommendations):
    match = process.extractOne(movie_name, df_movies_genre["tag"])
    if match is not None:
        idx = match[2]
        print("Movie Selected: ", df_movies_genre["tag"].iloc[idx], "Index: ", idx)
        print("Searching for recommendations....")
        distances, indices = model.kneighbors(sparse_matrix[idx].reshape(1, -1), n_neighbors=n_recommendations)
        for i in indices:
            if i != idx and 0 <= i < len(df_movies_genre):  # Kontroll för att undvika ogiltiga index
                print(df_movies_genre["tag"].iloc[i])
            else:
                print("Invalid index:", i)
    else:
        print("Movie not found:", movie_name)

# Använd funktionen för att göra rekommendationer för en specifik film inom den valda genren
recommender_genre('Notebook', model_knn, 20)
